# Long text summarization using LCEL chains on Langchain with Bedrock APIs

> *This notebook should work well with the **`Data Science 3.0`** kernel in SageMaker Studio*

## Overview
When we work with large documents, we can face some challenges as the input text might not fit into the model context length, or the model hallucinates with large documents, or, out of memory errors, etc.

To solve those problems, we are going to show a solution that is based on the concept of chunking and chaining prompts. This solution is leveraging [LangChain](https://python.langchain.com/docs/get_started/introduction.html) which is a popular framework for developing applications powered by language models.

In this architecture:

1. A large document (or a giant file appending small ones) is loaded
1. Langchain utility is used to split it into multiple smaller chunks (chunking)
1. First chunk is sent to the model; Model returns the corresponding summary
1. Langchain gets next chunk and appends it to the returned summary and sends the combined text as a new request to the model; the process repeats until all chunks are processed
1. In the end, you have final summary based on entire content

### Use case
This approach can be used to summarize call transcripts, meetings transcripts, books, articles, blog posts, and other relevant content.

### Imports

In [1]:
import os
import sys
from langchain_aws import ChatBedrockConverse
from IPython.core.display import HTML
from IPython.display import display_markdown, Markdown
import boto3

HTML("<script>Jupyter.notebook.kernel.restart()</script>")

module_path = ".."
sys.path.append(os.path.abspath(module_path))

boto3_bedrock = boto3.client('bedrock-runtime')

textgen_llm = ChatBedrockConverse(
    model_id="amazon.nova-micro-v1:0",
    client=boto3_bedrock,
    max_tokens=None,
    temperature=0.5
)


### Load shareholder letter

We will be following a process similar to lab 02 in this summarization section. First, let us load the 2022 Amazon shareholder letter

In [2]:
shareholder_letter = "./letters/2022-letter.txt"

with open(shareholder_letter, "r") as file:
    letter = file.read()

In [3]:
len(letter.split(' '))

5084

In [15]:
from langchain.text_splitter import RecursiveCharacterTextSplitter
text_splitter = RecursiveCharacterTextSplitter(
    separators=["\n\n", "\n"], chunk_size=8096, chunk_overlap=100
)

docs = text_splitter.create_documents([letter])

In [7]:
from langchain.prompts import PromptTemplate
from langchain.output_parsers import XMLOutputParser
from langchain.schema.output_parser import StrOutputParser


xml_parser = XMLOutputParser(tags=['insight'])
str_parser = StrOutputParser()

prompt = PromptTemplate(
    template="""
    
    Human:
    {instructions} : \"{document}\"
    Format help: {format_instructions}.
    Assistant:""",
    input_variables=["instructions","document"],
    partial_variables={"format_instructions": xml_parser.get_format_instructions()},
)

insight_chain = prompt | textgen_llm | StrOutputParser()

In [8]:
print(f"Number of Documents {len(docs)}")

Number of Documents 27


# Option 1. Manually process insights, then summarize

In [9]:
%%time
insights=[]
for i in range(len(docs)):
    insights.append(
        insight_chain.invoke({
        "instructions":"Provide Key insights from the following text",
        "document": {docs[i].page_content}
    }))

CPU times: user 118 ms, sys: 0 ns, total: 118 ms
Wall time: 26.1 s


In [10]:
str_parser = StrOutputParser()

prompt = PromptTemplate(
    template="""
    
    Human:
    {instructions} : \"{document}\"
    Assistant:""",
    input_variables=["instructions","document"]
)

summary_chain = prompt | textgen_llm | StrOutputParser()

In [11]:
%%time
display_markdown(Markdown(summary_chain.invoke({
        "instructions":"You will be provided with multiple sets of insights. Compile and summarize these insights and provide key takeaways in one concise paragraph. Do not use the original xml tags. Just provide a paragraph with your compiled insights.",
        "document": {'\n'.join(insights)}
    })))

The compiled insights from Amazon's various reports and statements highlight the company's resilience, innovation, and strategic growth amidst macroeconomic challenges. Despite a difficult macroeconomic environment in 2022, Amazon managed to grow demand and innovate across its largest businesses, improving customer experience both in the short and long term. The company has a history of successfully navigating economic downturns, such as during the dot-com crash and the 2008-2009 recession, by streamlining costs and prioritizing long-term customer experience. Amazon's AWS division, which received substantial investment in 2008, has grown to an $85 billion annual revenue business, underscoring the importance of long-term investments. Amazon continues to expand its footprint globally and into new markets, including doubling its fulfillment centers and accelerating its last-mile network. The company is also innovating in areas like machine learning and healthcare, with initiatives like the Kuiper satellite system and the acquisition of One Medical. Amazon Business has thrived by leveraging its ecommerce and logistics capabilities to serve the business market. The company remains optimistic about its future, driven by a culture of innovation, strategic investments, and a commitment to customer-focused, long-term growth.

CPU times: user 7.53 ms, sys: 95 μs, total: 7.63 ms
Wall time: 1.93 s


# Option 2. Use Map reduce pattern on Langchain

In [12]:
from langchain.chains.summarize import load_summarize_chain
summary_chain = load_summarize_chain(llm=textgen_llm, chain_type="map_reduce", verbose=False, token_max=1024)

In [18]:
%%time
display_markdown(Markdown(summary_chain.invoke(docs)['input_documents'][0].page_content))

As I sit down to write my second annual shareholder letter as CEO, I find myself optimistic and energized by what lies ahead for Amazon. Despite 2022 being one of the harder macroeconomic years in recent memory, and with some of our own operating challenges to boot, we still found a way to grow demand (on top of the unprecedented growth we experienced in the first half of the pandemic). We innovated in our largest businesses to meaningfully improve customer experience short and long term. And, we made important adjustments in our investment decisions and the way in which we’ll invent moving forward, while still preserving the long-term investments that we believe can change the future of Amazon for customers, shareholders, and employees.

While there were an unusual number of simultaneous challenges this past year, the reality is that if you operate in large, dynamic, global market segments with many capable and well-funded competitors (the conditions in which Amazon operates all of its businesses), conditions rarely stay stagnant for long.

In the 25 years I’ve been at Amazon, there has been constant change, much of which we’ve initiated ourselves. When I joined Amazon in 1997, we had booked $15M in revenue in 1996, were a books-only retailer, did not have a third-party marketplace, and only shipped to addresses in the US. Today, Amazon sells nearly every physical and digital retail item you can imagine, with a vibrant third-party seller ecosystem that accounts for 60% of our unit sales, and reaches customers in virtually every country around the world. Similarly, building a business around a set of technology infrastructure services in the cloud was not obvious in 2003 when we started pursuing AWS, and still wasn’t when we launched our first services in 2006. Having virtually every book at your fingertips in 60 seconds, and then being able to store and retrieve them on a lightweight digital reader was not “a thing” yet when we launched Kindle in 2007, nor was a voice-driven personal assistant like Alexa (launched in 2014) that you could use to access entertainment, control your smart home, shop, and retrieve all sorts of information.

There have also been times when macroeconomic conditions or operating inefficiencies have presented us with new challenges. For instance, in the 2001 dot-com crash, we had to secure letters of credit to buy inventory for the holidays, streamline costs to deliver better profitability for the business, yet still prioritized the long-term customer experience and business we were trying to build (if you remember, we actually lowered prices in most of our categories during that tenuous 2001 period). You saw this sort of balancing again in 2008-2009 as we endured the recession provoked by the mortgage-backed securities financial crisis. We took several actions to manage the cost structure and efficiency of our Stores business, but we also balanced this streamlining with investment in customer experiences that we believed could be substantial future businesses with strong returns for shareholders. In 2008, AWS was still a fairly small, fledgling business. We knew we were on to something, but it still required substantial capital investment. There were voices inside and outside of the company questioning why Amazon (known mostly as an online retailer then) would be investing so much in cloud computing. But, we knew we were inventing something special that could create a lot of value for customers and Amazon in the future. We had a head start on potential competitors; and if anything, we wanted to accelerate our pace of innovation. We made the long-term decision to continue investing in AWS. Fifteen years later, AWS is now an $85B annual revenue run rate business, with strong profitability, that has transformed how customers from start-ups to multinational companies to public sector organizations manage their technology infrastructure. Amazon would be a different company if we’d slowed investment in AWS during that 2008-2009 period.

Change is always around the corner. Sometimes, you proactively invite it in, and sometimes it just comes a-knocking. But, when you see it’s coming, you have to embrace it. And, the companies that do this well over a long period of time usually succeed. I’m optimistic about our future prospects because I like the way our team is responding to the changes we see in front of us.

Over the last several months, we took a deep look across the company, business by business, invention by invention, and asked ourselves whether we had conviction about each initiative’s long-term potential to drive enough revenue, operating income, free cash flow, and return on invested capital. In some cases, it led to us shuttering certain businesses. For instance, we stopped pursuing physical store concepts like our Bookstores and 4 Star stores, closed our Amazon Fabric and Amazon Care efforts, and moved on from some newer devices where we didn’t see a path to meaningful returns. In other cases, we looked at some programs that weren’t producing the returns we’d hoped (e.g. free shipping for all online grocery orders over $35) and amended them. We also reprioritized where to spend our resources, which ultimately led to the hard decision to eliminate 27,000 corporate roles. There are a number of other changes that we’ve made over the last several months to streamline our overall costs, and like most leadership teams, we’ll continue to evaluate what we’re seeing in our business and proceed adaptively.

We also looked hard at how we were working together as a team and asked our corporate employees to come back to the office at least three days a week, beginning in May. During the pandemic, our employees rallied to get work done from home and did everything possible to keep up with the unexpected circumstances that presented themselves. It was impressive and I’m proud of the way our collective team came together to overcome unprecedented challenges for our customers, communities, and business. But, we don’t think it’s the best long-term approach. We’ve become convinced that collaborating and inventing is easier and more effective when we’re working together and learning from one another in person. The energy and riffing on one another’s ideas happen more freely, and many of the best Amazon inventions have had their breakthrough moments from people staying behind after a meeting and working through ideas on a whiteboard, or continuing the conversation on the walk back from a meeting, or just popping by a teammate’s office later that day with another thought. Invention is often messy. It wanders and meanders and marinates. Serendipitous interactions help it, and there are more of those in-person than virtually. It’s also significantly easier to learn, model, practice, and strengthen our culture when we’re in the office together most of the time and surrounded by our colleagues. Innovation and our unique culture have been incredibly important in our first 29 years as a company, and I expect it will be comparably so in the next 29.

A critical challenge we’ve continued to tackle is the rising cost to serve in our Stores fulfillment network (i.e. the cost to get a product from Amazon to a customer)—and we’ve made several changes that we believe will meaningfully improve our fulfillment costs and speed of delivery.

CPU times: user 22.6 ms, sys: 0 ns, total: 22.6 ms
Wall time: 6.33 s
